In [70]:
%matplotlib inline
import sys
#print(sys.path)
sys.path.append("/Users/raroito/PycharmProjects/facial_expression_recognition/src/")

from utils import ModelImporter, Preprocessing
import torch
import pandas as pd
import numpy as np

# Analize Test Data Results

In [46]:
pre = Preprocessing('fer2013')
pre.load_data(filename='test_public_norm.csv', name='test')

X_df = pre.get(name='test').drop(columns=['emotion'])
y_df = pre.get(name='test')['emotion']

In [47]:
dtype = torch.float
device = torch.device("cpu")

model_name = 'cnn_simple'

m_importer = ModelImporter('fer2013')

n_classes = 7
n_epochs = 100
model = m_importer.load_nn_model(model_name, 0, n_classes, n_epochs)

load model CnnSimple(
  (conv1): Conv2d(1, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8640, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=7, bias=True)
)


In [48]:
X_test = model.reshape_data(torch.tensor(X_df.values, device=device, dtype=dtype))
y_test = torch.tensor(y_df.values, device=device, dtype=torch.long)

In [49]:
y_pred_prob = model(X_test)

In [50]:
y_pred = y_pred_prob.argmax(1)

In [51]:
y_pred_prob

tensor([[8.7998e-01, 9.6759e-17, 4.6826e-01,  ..., 6.3102e-06, 6.8502e-01,
         6.6125e-06],
        [8.1781e-01, 1.0052e-15, 6.7618e-01,  ..., 9.7284e-01, 5.0815e-05,
         6.7792e-02],
        [5.6206e-01, 4.8016e-13, 8.4401e-01,  ..., 9.7630e-01, 4.3635e-04,
         7.5559e-01],
        ...,
        [9.6324e-01, 1.0108e-17, 9.5018e-01,  ..., 9.9998e-01, 4.9806e-04,
         1.0000e+00],
        [9.9724e-01, 5.7961e-15, 4.1593e-01,  ..., 7.2267e-01, 2.5122e-07,
         9.5401e-01],
        [6.7508e-01, 3.9075e-12, 3.8478e-01,  ..., 9.1960e-01, 8.1901e-05,
         9.9923e-01]], grad_fn=<SigmoidBackward>)

In [52]:
y_pred

tensor([3, 3, 4,  ..., 6, 0, 6])

In [53]:
results = pd.DataFrame()

In [54]:
results['y_test'] = y_test
results['y_pred'] = y_pred
results['right'] = (y_pred == y_test).float()
results['ones'] = 1

In [55]:
results.tail()

,y_test,y_pred,right,ones
3584,4,3,0.0,1
3585,3,3,1.0,1
3586,4,6,0.0,1
3587,4,0,0.0,1
3588,4,6,0.0,1


In [56]:
report = results.groupby('y_test').sum()
report

,y_pred,right,ones
y_test,,,
0,1396,121.0,467
1,142,0.0,56
2,1680,98.0,496
3,2900,623.0,895
4,2310,161.0,653
5,1735,222.0,415
6,2583,295.0,607


In [57]:
report['%'] = report['right']/report['ones']
report

,y_pred,right,ones,%
y_test,,,,
0,1396,121.0,467,0.259101
1,142,0.0,56,0.000000
2,1680,98.0,496,0.197581
3,2900,623.0,895,0.696089
4,2310,161.0,653,0.246554
5,1735,222.0,415,0.534940
6,2583,295.0,607,0.485997


In [71]:
report['ones'].sum()

3589

In [58]:
#class 3 is somhow well recognized (also is the class with more samples) 
#whereas not a single sample of class 2 was properly detected

# Analize the Training Dataset

In [59]:
pre = Preprocessing('fer2013')
pre.load_data(filename='train_norm.csv', name='train')

X_df = pre.get(name='train').drop(columns=['emotion'])
y_df = pre.get(name='train')['emotion']

In [60]:
X_train= model.reshape_data(torch.tensor(X_df.values, device=device, dtype=dtype))
y_train = torch.tensor(y_df.values, device=device, dtype=torch.long)

In [61]:
y_pred_prob = model(X_train)

In [62]:
y_pred = y_pred_prob.argmax(1)

In [63]:
y_pred_prob

tensor([[9.4279e-01, 7.7489e-13, 7.2470e-02,  ..., 9.3389e-01, 3.4865e-04,
         2.0234e-01],
        [9.9909e-01, 1.4779e-13, 5.9159e-01,  ..., 9.3531e-01, 2.5739e-04,
         2.2301e-02],
        [2.1908e-03, 4.5221e-16, 9.6216e-03,  ..., 8.4227e-02, 4.5770e-01,
         3.3807e-01],
        ...,
        [4.7102e-01, 4.2740e-13, 5.6189e-01,  ..., 9.9913e-01, 2.0440e-04,
         5.5143e-01],
        [1.6083e-03, 5.7853e-12, 5.9794e-01,  ..., 3.9897e-03, 9.7383e-01,
         4.0441e-01],
        [3.3839e-01, 6.5697e-15, 6.9736e-01,  ..., 6.0949e-02, 2.2432e-01,
         2.9055e-01]], grad_fn=<SigmoidBackward>)

In [64]:
y_pred

tensor([0, 0, 5,  ..., 4, 5, 2])

In [65]:
y_train

tensor([0, 0, 2,  ..., 4, 0, 4])

In [66]:
results_train['y_train'] = y_train
results_train['y_pred'] = y_pred
results_train['right'] = (y_pred == y_train).float()
results_train['ones'] = 1

In [67]:
report_train = results_train.groupby('y_train').sum()
report_train

,y_pred,right,ones
y_train,,,
0,10777,1336.0,3995
1,1302,0.0,436
2,13513,839.0,4097
3,22852,5273.0,7215
4,17240,1232.0,4830
5,13354,1793.0,3171
6,22026,2568.0,4965


In [68]:
report_train['%'] = report_train['right']/report_train['ones']
report_train

,y_pred,right,ones,%
y_train,,,,
0,10777,1336.0,3995,0.334418
1,1302,0.0,436,0.000000
2,13513,839.0,4097,0.204784
3,22852,5273.0,7215,0.730839
4,17240,1232.0,4830,0.255072
5,13354,1793.0,3171,0.565437
6,22026,2568.0,4965,0.517221


In [ ]:
#similar results with the train data as with the 